In [13]:
from common import *

cursor = connect()

# INSERT Trigger

创建触发器函数
```sql
CREATE OR REPLACE FUNCTION trigger_function()
   RETURNS TRIGGER
   LANGUAGE PLPGSQL
AS
$$
BEGIN
   -- trigger logic
   -- ...
   RETURN NEW;
END;
$$
```

为表创建触发器
```sql
CREATE TRIGGER trigger_name
BEFORE INSERT
ON table_name
FOR EACH {ROW | STATEMENT}
EXECUTE FUNCTION trigger_function();
```

In [14]:
sql = """
CREATE TABLE inventory_t(
    product_id INT PRIMARY KEY,
    quantity INT NOT NULL DEFAULT 0
);

CREATE TABLE inventory_stat(
    total_qty INT
);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a992194710>

In [15]:
sql = """
CREATE OR REPLACE FUNCTION update_total_qty()
RETURNS TRIGGER
LANGUAGE PLPGSQL
AS
$$
DECLARE
   p_row_count INT;
BEGIN
   SELECT COUNT(*) FROM inventory_stat
   INTO p_row_count;
   
   IF p_row_count > 0 THEN
      UPDATE inventory_stat 
      SET total_qty = total_qty + NEW.quantity;
   ELSE
      INSERT INTO inventory_stat(total_qty)
      VALUES(new.quantity);
   END IF;
   RETURN NEW;
END;
$$;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a992194710>

In [16]:
sql = """
CREATE TRIGGER inventory_before_insert
BEFORE INSERT
ON inventory_t
FOR EACH ROW
EXECUTE FUNCTION update_total_qty();

INSERT INTO inventory_t(product_id, quantity)
VALUES(1, 100)
RETURNING *;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a992194710>

In [17]:
sql = """
SELECT * FROM inventory_stat;
"""
run_sql(cursor, sql)

   total_qty
0        100


In [18]:
sql = """
INSERT INTO inventory_t(product_id, quantity)
VALUES(2, 200)
RETURNING *;
"""
cursor.execute(sql)

<psycopg.Cursor [TUPLES_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a992194710>

In [19]:
sql = """
SELECT * FROM inventory_stat;
"""
run_sql(cursor, sql)

   total_qty
0        300
